In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW tv_orders_fixed
AS
select value,
regexp_replace(value,'"order_date": (\\d{4}-\\d{2}-\\d{2})','"order_date": "\$1"')
as fixed_value 
from 
gizmobox.bronze.v_orders;

In [0]:
%sql
select * from tv_orders_fixed limit 10

In [0]:
%sql
select from_json(fixed_value,'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') from tv_orders_fixed limit 1

In [0]:
%sql
DROP TABLE gizmobox.silver.orders_json;
CREATE TABLE IF NOT EXISTS gizmobox.silver.orders_json
as 
select from_json(fixed_value,'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value from tv_orders_fixed ;